In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



#configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#import previous clean data set
df= pd.read_csv("../WeatherPy/WeatherPy_removed_DidNotFind_Rows.csv")
new_df=pd.DataFrame(df)


#update temp to F
kelvin = new_df["Max Temp"]
new_df["Max Temp"] = 1.8 * (kelvin - 273) + 32

#update wind to MPH
mph =2.23694
new_df["Wind Speed"] =new_df["Wind Speed"]*mph




fig=gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(
    new_df(['Lat', 'Lng']), weights=new_df['Humidity'], max_intensity=30, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig


In [3]:
fig=gmaps.figure() 
heatmap_layer=gmaps.heatmap_layer( 
    new_df[['Lat', 'Lng']], weights=new_df['Humidity'], 
     point_radius=5.0) 
fig.add_layer(heatmap_layer) 
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# get a managable amount of city information for hotel searches

#new_df_hotels= new_df.loc[(new_df["Max Temp"]>70) & (new_df["Max Temp"]<80) & (new_df["Wind Speed"]<5) & (new_df["Cloudiness"]==0)]
new_df_hotels= new_df.loc[(new_df["Max Temp"]>70) & (new_df["Max Temp"]<73) & (new_df["Country"]=='US')]
len(new_df_hotels)

5

In [5]:
#Show a record of small dataframe for reference
new_df_hotels

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
234,259,fallon,39.47,-118.78,72.266,35,40,6.934514,US,1602699694
255,283,freeport,40.66,-73.58,70.268,40,1,11.408394,US,1602699760
308,339,saint george,37.10,-113.58,70.070,15,1,3.780429,US,1602699866
404,439,pacific grove,36.62,-121.92,72.266,53,1,4.697574,US,1602699990
424,460,brigantine,39.41,-74.36,70.070,43,1,8.052984,US,1602700016


In [6]:
#crreate new dataframe for hotel information
Hotel_df = pd.DataFrame(new_df_hotels["City"])

Hotel_df["Hotel Lat"] =''
Hotel_df["Hotel Lng"] =''
Hotel_df["Hotel Name"] =''
Hotel_df["Hotel Country"] =new_df_hotels['Country']


#Hotel_df.drop('index',axis=1)
Hotel_df1 = pd.DataFrame(Hotel_df,columns=["City","Hotel Lat","Hotel Lng","Hotel Name","Hotel Country"])

Hotel_df1

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
234,fallon,,,,US
255,freeport,,,,US
308,saint george,,,,US
404,pacific grove,,,,US
424,brigantine,,,,US


In [7]:
#drops the old index column 
Hotel_df1.reset_index(drop=True)

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
0,fallon,,,,US
1,freeport,,,,US
2,saint george,,,,US
3,pacific grove,,,,US
4,brigantine,,,,US


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_radius = 8000
target_type = "lodging"

#iterate of newly created dataframe to populate name, lat and long   
for index, row in new_df_hotels.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']   
    coords=f"{lat}, {lng}"

    params = {"location":  coords ,"radius": target_radius, "type": target_type, "key": g_key}    
    response_place = requests.get(base_url, params=params).json()
  
    Hotel_df1.loc[index, "Hotel Name"] =response_place["results"][0]["name"]
    Hotel_df1.loc[index, "Hotel Lat"] = response_place["results"][0]["geometry"]["location"]["lat"]
    Hotel_df1.loc[index, "Hotel Lng"] = response_place["results"][0]["geometry"]["location"]["lng"]
    

#prints out populated dataframe
print(Hotel_df1)


#print(new_df_hotels)
#print(json.dumps(response_place, indent=4, sort_keys=True))

              City Hotel Lat Hotel Lng                        Hotel Name  \
234         fallon   39.4746   -118.79  Best Western Fallon Inn & Suites   
255       freeport   40.6578  -73.6488     Best Western Mill River Manor   
308   saint george    37.089  -113.584       Best Western Plus Abbey Inn   
404  pacific grove   36.6256  -121.917                  Lovers Point Inn   
424     brigantine   39.3699  -74.4126                   Flagship Resort   

    Hotel Country  
234            US  
255            US  
308            US  
404            US  
424            US  


In [12]:
#generate dataframe for reference
Hotel_df1

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
234,fallon,39.4746,-118.79,Best Western Fallon Inn & Suites,US
255,freeport,40.6578,-73.6488,Best Western Mill River Manor,US
308,saint george,37.089,-113.584,Best Western Plus Abbey Inn,US
404,pacific grove,36.6256,-121.917,Lovers Point Inn,US
424,brigantine,39.3699,-74.4126,Flagship Resort,US


In [13]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.


#create location markers
latit=[]
long=[]

for lat in Hotel_df1['Hotel Lat']:
    #concat = f"{lat}, {long}"
    #concat_int = int()
    latit.append(lat)
    

for lng in Hotel_df1['Hotel Lng']:
    #concat = f"{lat}, {long}"
    #concat_int = int()
    long.append(lng)
 
#create location markers as a list
loc = list(zip(latit,long))

#narrow down the data in the markers
last_df=  Hotel_df1[["City","Hotel Name", "Hotel Country"]]



#create the list for data of the hotel
hotel_info= []

for city,hotelname,country in zip(last_df["City"],last_df["Hotel Name"], last_df["Hotel Country"]):
    concat = f"{city}, {hotelname}, {country}"
    hotel_info.append(concat)
   



#create the marker on top of the heatmap
marker_layer = gmaps.marker_layer(loc, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))